In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,GlobalAveragePooling1D,LSTM,Flatten,Dropout,Dense
from keras.preprocessing.text import one_hot
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
import numpy as np
import nltk

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('C:/deep learning/rnn/datasets/spam.csv')
df.shape

(5572, 2)

In [6]:
df.dropna().head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
X=df.drop('Category',axis=1)
X.head()

,Message
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
y=df['Category']


In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Category']= le.fit_transform(df['Category']) 
labels=df['Category']
labels.head()



0    0
1    0
2    1
3    0
4    0
Name: Category, dtype: int64

text preprocessing

In [11]:
messeges=X.copy()

In [68]:
#remove all punctuation and stopwords for pure data
d=messeges['Message'][0]
d

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [13]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucifer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
len(messeges)

5572

In [16]:
# it is a text preprocessing
corpus=[]
ps=PorterStemmer()
for i in range(0,len(messeges)):
    review=re.sub('[^a-zA-Z]',' ',messeges['Message'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [17]:
#one hot encoding
vocab=5000
encoded_docs=[one_hot(words,vocab) for words in corpus]

In [18]:
#pad sequence because differn sentence differnt lenght and embedding layer wants the sentence in same lenghth as output
paded_docs=pad_sequences(encoded_docs,30)

In [19]:
X=np.array(paded_docs)
y=np.array(labels)

In [20]:
embedding_vector=40
model=Sequential()
model.add(Embedding(vocab,embedding_vector,input_length=30))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
#prepare data for model
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=40)




In [23]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,verbose=1,batch_size=100)

C:\Users\Lucifer\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 4s 834us/step - loss: 0.3669 - accuracy: 0.8692 - val_loss: 0.1928 - val_accuracy: 0.9274
Epoch 2/10
4457/4457 [==============================] - 3s 613us/step - loss: 0.1183 - accuracy: 0.9717 - val_loss: 0.0845 - val_accuracy: 0.9776
Epoch 3/10
4457/4457 [==============================] - 3s 639us/step - loss: 0.0425 - accuracy: 0.9886 - val_loss: 0.0637 - val_accuracy: 0.9821
Epoch 4/10
4457/4457 [==============================] - 2s 547us/step - loss: 0.0232 - accuracy: 0.9939 - val_loss: 0.0675 - val_accuracy: 0.9848
Epoch 5/10
4457/4457 [==============================] - 2s 543us/step - loss: 0.0162 - accuracy: 0.9964 - val_loss: 0.0663 - val_accuracy: 0.9803
Epoch 6/10
4457/4457 [==============================] - 3s 576us/step - loss: 0.0100 - accuracy: 0.9978 - val_loss: 0.0867 - val_accuracy: 0.9812
Epoch 7/10
4457/4457 [==============================] - 3s 595us/step - loss

In [34]:
from sklearn.metrics import confusion_matrix
y_pre=model.predict_classes(X_test)

In [35]:
confusion_matrix(y_test,y_pre)

array([[961,  10],
       [ 14, 130]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pre)

0.97847533632287

In [96]:
df['Message'][5]

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"

In [103]:
#check the model accuracy with real data
sent=input()
input_msg=sent
corpus=[]
ps=PorterStemmer()
for i in range(0,1):
    review=re.sub('[^a-zA-Z]',' ',input_msg)
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
vocab=5000
encoded_docs=[one_hot(words,vocab) for words in corpus]
paded_docs=pad_sequences(encoded_docs,30)
msg=np.array(paded_docs)
print(msg)
model.predict(msg)


can you call later
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
  3157 3984]]


array([[0.00987868]], dtype=float32)

In [98]:
#adding dropout in modal to improve the accuracy
embedding_vector=40
model1=Sequential()
model1.add(Embedding(vocab,embedding_vector,input_length=30))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,verbose=1,batch_size=100)

C:\Users\Lucifer\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 3s 746us/step - loss: 0.3778 - accuracy: 0.8629 - val_loss: 0.1836 - val_accuracy: 0.9480
Epoch 2/10
4457/4457 [==============================] - 3s 604us/step - loss: 0.1326 - accuracy: 0.9697 - val_loss: 0.0922 - val_accuracy: 0.9749
Epoch 3/10
4457/4457 [==============================] - 3s 598us/step - loss: 0.0452 - accuracy: 0.9888 - val_loss: 0.0745 - val_accuracy: 0.9785
Epoch 4/10
4457/4457 [==============================] - 3s 633us/step - loss: 0.0269 - accuracy: 0.9935 - val_loss: 0.0752 - val_accuracy: 0.9812
Epoch 5/10
4457/4457 [==============================] - 3s 592us/step - loss: 0.0173 - accuracy: 0.9964 - val_loss: 0.0806 - val_accuracy: 0.9776
Epoch 6/10
4457/4457 [==============================] - 3s 570us/step - loss: 0.0159 - accuracy: 0.9964 - val_loss: 0.0845 - val_accuracy: 0.9767
Epoch 7/10
4457/4457 [==============================] - 3s 570us/step - loss

In [102]:
from sklearn.metrics import confusion_matrix
y_pre=model1.predict_classes(X_test)
confusion_matrix(y_test,y_pre)

array([[967,   4],
       [ 24, 120]], dtype=int64)

In [103]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pre)

0.9748878923766816